In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.jmt.*
import jline.solvers.mva.*
import jline.util.matrix.*

In [ ]:
// Fork-Join with Class Switching
val model = Network("model")

In [ ]:
// Create network
val delay = Delay(model, "Delay1")
val delay2 = Delay(model, "Delay2")
val queue1 = Queue(model, "Queue1", SchedStrategy.PS)
val queue2 = Queue(model, "Queue2", SchedStrategy.PS)
val fork = Fork(model, "Fork")
val join = Join(model, "Join", fork)

val jobclass1 = ClosedClass(model, "class1", 10, delay, 0)
val jobclass2 = ClosedClass(model, "class2", 10, delay, 0)

In [ ]:
// Service configurations
queue1.setService(jobclass1, Exp(1.0))
queue2.setService(jobclass1, Exp(1.0))
delay.setService(jobclass1, Exp(0.5))
delay.setService(jobclass2, Exp(0.5))
delay2.setService(jobclass2, Exp(2.0))

In [ ]:
// Set routing matrix with class switching
val P = model.initRoutingMatrix()

// Class switching routing: jobclass1 -> jobclass2 -> jobclass1
P.set(jobclass1, jobclass2, delay, delay2, 1.0)    // Class switch: class1 -> class2 at delay
P.set(jobclass2, jobclass1, delay2, fork, 1.0)     // Class switch: class2 -> class1 at delay2

// Fork-join routing for class1
P.set(jobclass1, jobclass1, fork, queue1, 1.0)
P.set(jobclass1, jobclass1, fork, queue2, 1.0)
P.set(jobclass1, jobclass1, queue1, join, 1.0)
P.set(jobclass1, jobclass1, queue2, join, 1.0)
P.set(jobclass1, jobclass1, join, delay, 1.0)

model.link(P)

In [ ]:
// Solve with multiple methods
val solvers = mutableListOf<NetworkSolver>()

solvers.add(JMT(model, "seed", 23000))
solvers.add(MVA(model))

for ((i, solver) in solvers.withIndex()) {
    println("SOLVER ${i+1}: ${solver.name}")
    try {
        val avgTable = solver.avgTable
        avgTable.print()
    } catch (e: Exception) {
        println("Error: ${e.message}")
    }
}

This pre-fork class switching model demonstrates:

1. **Pre-Fork Class Switching Chain**:
   - **Phase 1**: Class1 jobs at Delay1 (rate 0.5)
   - **Phase 2**: Switch to Class2 at Delay2 (rate 2.0)
   - **Phase 3**: Switch back to Class1 before fork
   - **Phase 4**: Normal fork-join processing as Class1

2. **Sequential Class Transformation**:
   - Jobs cycle through: Class1 → Class2 → Class1 → Fork-Join
   - Class switching occurs during delay/preprocessing phases
   - Fork-join operations only see Class1 jobs

3. **Service Rate Differentiation**:
   - **Delay1**: Rate 0.5 (slow initial processing)
   - **Delay2**: Rate 2.0 (fast intermediate processing)
   - **Queue1/Queue2**: Rate 1.0 (moderate parallel processing)

4. **Population Distribution**:
   - 10 jobs each class initially
   - Class2 acts as intermediate transformation stage
   - Effective Class1 population for fork-join varies dynamically

5. **Performance Implications**:
   - **Delay1 bottleneck**: Slowest rate (0.5) limits overall throughput
   - **Fast transformation**: Delay2 (rate 2.0) processes quickly
   - **Fork-join amplification**: Each job creates parallel tasks

6. **Queueing Behavior**:
   - Jobs accumulate at Delay1 due to slow processing
   - Delay2 processes jobs faster than they arrive
   - Fork-join creates parallel processing load

The model represents:
- Multi-stage manufacturing with job preparation phases
- Software systems with preprocessing and parallel execution
- Document processing with classification and parallel handling
- Network protocols with header processing and parallel routing

Key insight: Pre-fork class switching allows for complex job preparation workflows before parallel processing begins, enabling different service characteristics during preparation versus execution phases.

**Performance Pattern**: The system throughput is limited by the slowest preparation stage (Delay1), regardless of parallel processing capacity in the fork-join section.